# Tuning of Hyperparameters in Simple Self-EFficacy Learner Model

This notebook includes code from iterating on the Self-Efficacy Learner Behavior Model including hyperparameters for generating the set of learners in the population.

In [1]:
import sys
sys.path.append("../")

In [2]:
import logging
import random
import uuid
import os

from tutor.domain import Domain
from tutor.curriculum import SimpleCurriculum
from tutor.tutor import SimpleTutor
from tutor.action import Attempt, HintRequest
from learner.selfeff_learner import SelfEfficacyLearner
from simulate.simple_tutor_simulation import SimpleTutorSimulation
from simulate.simulation import Simulation
from simulate.self_eff_simulation import SelfEffSimulation
from log_db import mongo
from log_db.curriculum_mapper import DB_Curriculum_Mapper

In [3]:
logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("main")

In [4]:
# Get path to current project directory
cwd = os.path.abspath(".")
base_dir = os.path.abspath(os.path.join(cwd, os.pardir))
logger.debug("Base directory for the project:\n%s" % base_dir)

DEBUG:main:Base directory for the project:
/Users/stevenda/Sandbox/MotivSim


In [5]:
# Setup connection to database
data_out = "sim-%s" % str(uuid.uuid4())
data_path = os.path.join(base_dir,"test", "data", data_out)
logger.info("Writing simulation results to directory: %s" % data_path)
db_name = "motivsim"
db_params  = mongo.get_db_params(db_name)
logger.info("got db params: %s" % str(db_params))
db_util = mongo.Data_Utility(data_path, db_params)
db = db_util.db

INFO:main:Writing simulation results to directory: /Users/stevenda/Sandbox/MotivSim/test/data/sim-f4d8258e-f6ea-4eb9-b64a-e441f432a919
INFO:main:got db params: {'settingId': 'motivsim', 'url': 'localhost', 'port': '27017', 'name': 'motivsim', 'user': '', 'pswd': ''}


In [6]:
logger.info("Clearing database")
db_util.clear_db()

INFO:main:Writing simulation results to directory: ../test/data/sim-a8cd29da-ad61-434b-bd1e-d942a1e112ff
INFO:main:got db params: {'name': u'motivsim', 'url': u'localhost', 'settingId': 'motivsim', 'user': u'', 'pswd': u'', 'port': u'27017'}
INFO:main:Clearing database
DEBUG:log_db.mongo:all collections in db: [u'curriculum', u'sections', u'problems', u'curriculums', u'units', u'decisions', u'students', u'tutor_events', u'steps', u'kcs']
DEBUG:log_db.mongo:Clearing all documents from collection curriculum
DEBUG:log_db.mongo:Clearing all documents from collection sections
DEBUG:log_db.mongo:Clearing all documents from collection problems
DEBUG:log_db.mongo:Clearing all documents from collection curriculums
DEBUG:log_db.mongo:Clearing all documents from collection units
DEBUG:log_db.mongo:Clearing all documents from collection decisions
DEBUG:log_db.mongo:Clearing all documents from collection students
DEBUG:log_db.mongo:Clearing all documents from collection tutor_events
DEBUG:log_db.mo

## Generating Learners

### Setting up Simulation

In [6]:
# Generating domain
domain = Simulation.gen_domain(30)
db.kcs.insert_many([kc.__dict__ for kc in domain.kcs])

INFO:simulate.simulation:Generating a new domain
DEBUG:tutor.domain:generate kcs for domain: 22341b84-b074-4a6f-b663-99627c4dd03e
DEBUG:tutor.domain:Generating kc #0
DEBUG:tutor.domain:KC: pl0: 0.119413	pt: 0.036456	pg: 0.386065	ps: 0.213162	mtime: 14.855173	 sdtime: 3.713793
DEBUG:tutor.domain:Generating kc #1
DEBUG:tutor.domain:KC: pl0: 0.338241	pt: 0.050448	pg: 0.357938	ps: 0.257711	mtime: 7.771944	 sdtime: 1.942986
DEBUG:tutor.domain:Generating kc #2
DEBUG:tutor.domain:KC: pl0: 0.376569	pt: 0.073837	pg: 0.406056	ps: 0.177365	mtime: 9.842890	 sdtime: 2.460723
DEBUG:tutor.domain:Generating kc #3
DEBUG:tutor.domain:KC: pl0: 0.318793	pt: 0.081292	pg: 0.296847	ps: 0.231383	mtime: 19.941786	 sdtime: 4.985446
DEBUG:tutor.domain:Generating kc #4
DEBUG:tutor.domain:KC: pl0: 0.262951	pt: 0.059025	pg: 0.371891	ps: 0.209476	mtime: 11.888141	 sdtime: 2.972035
DEBUG:tutor.domain:Generating kc #5
DEBUG:tutor.domain:KC: pl0: 0.156623	pt: 0.078379	pg: 0.338483	ps: 0.210887	mtime: 21.224216	 sdtime:

In [7]:
# Generating the Curriculum
curric = Simulation.gen_curriculum(domain, 1, 3, 20)
curric_util = DB_Curriculum_Mapper(db_params)
curric_util.write_to_db(curric)

INFO:simulate.simulation:Generating Curriculum with given domain
DEBUG:tutor.curriculum:Generating curriculum for <class 'tutor.curriculum.SimpleCurriculum'>
DEBUG:tutor.curriculum:kcs by unit: [30]
DEBUG:tutor.curriculum:assigning kcs: [<tutor.domain.KC object at 0x1152c4520>, <tutor.domain.KC object at 0x107689c10>, <tutor.domain.KC object at 0x1152c4ca0>, <tutor.domain.KC object at 0x107689a60>, <tutor.domain.KC object at 0x1152c4490>, <tutor.domain.KC object at 0x1152c4b80>, <tutor.domain.KC object at 0x1152c4400>, <tutor.domain.KC object at 0x1152c4610>, <tutor.domain.KC object at 0x1152c44c0>, <tutor.domain.KC object at 0x1152ba250>, <tutor.domain.KC object at 0x1152ba0a0>, <tutor.domain.KC object at 0x1152c4d30>, <tutor.domain.KC object at 0x1152c4e80>, <tutor.domain.KC object at 0x1152c4670>, <tutor.domain.KC object at 0x1152c4ac0>, <tutor.domain.KC object at 0x1152c4fa0>, <tutor.domain.KC object at 0x1152c4d00>, <tutor.domain.KC object at 0x1152c4f10>, <tutor.domain.KC object 

DEBUG:tutor.curriculum:New step mean time: 11.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 12.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 14.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 17.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 6.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 21.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 20.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 14.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 14.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 6.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 20.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 15.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 19.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 7.000000	 sd time: 2.00

DEBUG:tutor.curriculum:New step mean time: 22.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 22.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 30.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 17.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 32.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 13.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 15.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 18.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 33.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 23.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 33.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:Section problems before add: 0
DEBUG:tutor.curriculum:Section problems after add: 200
DEBUG:tutor.curriculum:New step mean time: 18.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New 

DEBUG:tutor.curriculum:New step mean time: 6.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 20.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 9.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 22.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 36.000000	 sd time: 9.000000
DEBUG:tutor.curriculum:New step mean time: 11.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 11.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 11.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 33.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 6.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 14.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 32.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 18.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 12.000000	 sd time: 3.00

DEBUG:tutor.curriculum:New step mean time: 28.000000	 sd time: 7.000000
DEBUG:tutor.curriculum:New step mean time: 17.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 23.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 17.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 11.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 30.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 23.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 19.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 19.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 28.000000	 sd time: 7.000000
DEBUG:tutor.curriculum:New step mean time: 24.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 20.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 32.000000	 sd time: 8.000000
DEBUG:tutor.curriculum:New step mean time: 22.000000	 sd time: 6

DEBUG:tutor.curriculum:New step mean time: 24.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 25.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 24.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 4.000000	 sd time: 1.000000
DEBUG:tutor.curriculum:New step mean time: 24.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 4.000000	 sd time: 1.000000
DEBUG:tutor.curriculum:New step mean time: 13.000000	 sd time: 3.000000
DEBUG:tutor.curriculum:New step mean time: 18.000000	 sd time: 4.000000
DEBUG:tutor.curriculum:New step mean time: 4.000000	 sd time: 1.000000
DEBUG:tutor.curriculum:New step mean time: 10.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 10.000000	 sd time: 2.000000
DEBUG:tutor.curriculum:New step mean time: 23.000000	 sd time: 6.000000
DEBUG:tutor.curriculum:New step mean time: 20.000000	 sd time: 5.000000
DEBUG:tutor.curriculum:New step mean time: 13.000000	 sd time: 3.00

### Initializing Learners

In [8]:
sys.version

'3.8.5 (default, Jul 21 2020, 10:42:08) \n[Clang 11.0.0 (clang-1100.0.33.17)]'

### Simulating Learners